In [6]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=4f02deb0c49e630e661dbf1880dfebc1b9aa2a5df9cee01c21554ffd51ff5956
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
from google.colab import files

uploaded = files.upload()

Saving train.csv to train.csv


In [35]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkFiles

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)


# Initialize Spark
spark = SparkSession.builder \
    .appName("Python Spark Practice") \
    .getOrCreate()

data = [123, 254, 133, 4, 5321, 556, 7567, 78, 69, 10, 100]
rdd = spark.sparkContext.parallelize(data)

# Sum
rdd_sum = rdd.reduce(lambda a, b: a + b)
print(f"Sum: {rdd_sum}")

# Count
rdd_count = rdd.count()
print(f"Count: {rdd_count}")

data = [
  ('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname", "middlename", "lastname", "dob", "gender", "salary"]
df = spark.createDataFrame(data=data, schema = columns)
df.show()

df.createOrReplaceTempView("people")
result = spark.sql("SELECT * FROM people WHERE salary > 3500")
result.show()

Sum: 14215
Count: 11
+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
+---------+----------+--------+----------+------+------+



In [39]:
from pyspark import SparkFiles
from pyspark.sql import SparkSession

# Create a DataFrame based on the CSV file
df = spark.read.csv(("train.csv"), header=True, inferSchema=True)

# Show the DataFrame
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+
|     1|CA-2017-152156|08/11/2017|11/11/2017|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset Col...|  261.96|
|     2|CA-2017-152156|0

In [40]:
# Print the schema of DataFrame
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)



In [41]:
df.select('Product Name','sales').show(10)

+--------------------+--------+
|        Product Name|   sales|
+--------------------+--------+
|Bush Somerset Col...|  261.96|
|Hon Deluxe Fabric...|  731.94|
|Self-Adhesive Add...|   14.62|
|Bretford CR4500 S...|957.5775|
|Eldon Fold 'N Rol...|  22.368|
|Eldon Expressions...|   48.86|
|          Newell 322|    7.28|
|Mitel 5320 IP Pho...| 907.152|
|DXL Angle-View Bi...|  18.504|
|Belkin F5C206VTEL...|   114.9|
+--------------------+--------+
only showing top 10 rows



In [49]:
df.groupBy("Category").count().sort("count",ascending=True).show()


+---------------+-----+
|       Category|count|
+---------------+-----+
|     Technology| 1813|
|      Furniture| 2078|
|Office Supplies| 5909|
+---------------+-----+



In [43]:
df.describe().show()

+-------+------------------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+---------------+----------+------------+--------------------+-----------------+
|summary|            Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|    City|  State|       Postal Code| Region|     Product ID|  Category|Sub-Category|        Product Name|            Sales|
+-------+------------------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+---------------+----------+------------+--------------------+-----------------+
|  count|              9800|          9800|      9800|      9800|          9800|       9800|              9800|       9800|         9800|    9800|   9800|              9789|   9800|           9800|      9800|        9

In [44]:
df.describe('sales').show()

+-------+-----------------+
|summary|            sales|
+-------+-----------------+
|  count|             9800|
|   mean|235.2895627576727|
| stddev| 635.178384045279|
|    min|         10/Pack"|
|    max|           999.98|
+-------+-----------------+



In [45]:
df.crosstab('Ship Mode', 'Segment').show()

+-----------------+--------+---------+-----------+
|Ship Mode_Segment|Consumer|Corporate|Home Office|
+-----------------+--------+---------+-----------+
|      First Class|     755|      468|        278|
|         Same Day|     312|      114|        112|
|     Second Class|    1003|      589|        310|
|   Standard Class|    3031|     1782|       1046|
+-----------------+--------+---------+-----------+



In [46]:
df.drop('Sub-Category').columns

['Row ID',
 'Order ID',
 'Order Date',
 'Ship Date',
 'Ship Mode',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal Code',
 'Region',
 'Product ID',
 'Category',
 'Product Name',
 'Sales']

In [47]:
df.filter(df.Sales > 100).count()

3620

In [48]:
df.groupby('State').agg({'Sales': 'mean'}).show()

+--------------------+------------------+
|               State|        avg(Sales)|
+--------------------+------------------+
|                Utah|225.00076595744676|
|           Minnesota|  335.541011235955|
|                Ohio|167.02240681818168|
|              Oregon|145.21537288135588|
|            Arkansas|194.83372881355928|
|               Texas|176.23768121720886|
|        North Dakota|131.41571428571427|
|        Pennsylvania|208.14217967332127|
|         Connecticut|166.47246249999995|
|            Nebraska|201.27810810810803|
|             Vermont|  811.760909090909|
|              Nevada| 437.9711052631578|
|          Washington| 274.5479422680412|
|            Illinois| 168.0456924731181|
|            Oklahoma| 294.4946153846153|
|District of Columbia|286.50199999999995|
|            Delaware| 302.3381910112359|
|          New Mexico|129.28437837837836|
|       West Virginia|           302.456|
|            Missouri| 353.4083606557377|
+--------------------+------------